In [1]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import ast
import ts_gd as gd

In [4]:
def get_events_list():
    import requests

    params = (
        ('bk_name', 'liga_stavok'),
        ('sport', 'soccer'),
        ('token', '79b4fd9c0671e8ff0fb77d9f8361ec10'),
        ('live', '0'),
    )

    r = requests.get('http://api.oddscp.com:8111/markets', 
                     params=params)
    print(r.status_code)    
    
    return r.json()

# на входе - одна строчка
def prepair_bet_data(bet_json):
    from datetime import datetime
    
    df=pd.DataFrame.from_dict(bet_json, orient='index').T  
    df['meta']=df['meta'].apply(ast.literal_eval)
    df['description']=pd.json_normalize(df['meta'])['marketTitle']
    df['updated_at_msc']=df['updated_at'].apply(
        lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
    
    df=df[['bet', 'cf', 'description', 'updated_at', 'updated_at_msc', 'blocked']]
    
    return df

def get_event_data(event_id):
    import requests
    params = (
            ('token', '79b4fd9c0671e8ff0fb77d9f8361ec10'),
            ('live', '0'),
                )
    r = requests.get('http://api.oddscp.com:8111/events/'+str(event_id), params=params)
    return r.json()

In [5]:
get_events_list()

404


''

In [3]:
%%time
answ=get_events_list()
print(len(answ.keys()))
daily_df=pd.DataFrame()
for event_id in answ.keys():
    bets=answ[event_id]
    event_json=get_event_data(event_id)

    df=pd.DataFrame()
    for i in range(len(bets)):
        b=prepair_bet_data(bets[i])
        df=pd.concat([df, b])

    df['event_id']=event_id
    df['event_name']=event_json['event_name']
    df['team1']=event_json['team1']
    df['team2']=event_json['team2']
    df['league_name']=event_json['league_name']
    df['url']=event_json['direct_link']
    
    print(event_json['event_name'])
    
    daily_df=pd.concat([daily_df, df])
    


404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
len(daily_df)

39595

In [38]:
daily_df.sample(3)

,bet,cf,description,updated_at,updated_at_msc,blocked,event_id,event_name,team1,team2,league_name,url
0,TOTALS__OVER(3.5),3.1,Тотал \/ Основное время \/ Бол,1644815638,2022-02-14 05:13:58,0,LIGSC0C70D3904CA,Боруссия Дортмунд II vs Веен Висбаден,Боруссия Дортмунд II,Веен Висбаден,Германия. 3-я бундеслига,https://www.ligastavok.ru/bets/live/popular/germaniia-3-ia-bundesliga-id-268/3-ia-bundesliga-id-262/game-name-id-16611371
0,HALF_01__TOTALS__ODD,2.17,Чет\/нечет \/ 1-й тайм \/ Нечет,1644815478,2022-02-14 05:11:18,0,LIGSCC07488DD2F7,Фарул Констанца vs Арджеш,Фарул Констанца,Арджеш,Румыния. 1-я лига,https://www.ligastavok.ru/bets/live/popular/rumyniia-1-ia-liga-id-352/1-ia-liga-id-15561/game-name-id-16601645
0,HANDICAP__P1(0),1.3,Фора \/ Основное время \/ 1,1644815608,2022-02-14 05:13:28,0,LIGSCACA1BFE28E0,Арау vs Виль 1900,Арау,Виль 1900,Швейцария. Челлендж лига,https://www.ligastavok.ru/bets/live/popular/shveitcariia-chellendzh-liga-id-386/chellendzh-liga-id-7113/game-name-id-16611334


In [43]:
url='195K7SPoxAkAYSgp4wTVOTGIaHXCbhii7O3HM5nSmnjg'
gd.write(url, 'liga_stavok_bets!A2:ZZ90000', daily_df)

Ready
